# Simple Chain

* Perform several actions in a particular order

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]



In [3]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a curious fact about {politician}")

chain = prompt | chatModel | StrOutputParser()

**What does StrOutputParser do?**

* The StrOutputParser is a specific tool within LangChain that simplifies the output from these language models. It takes the complex or structured output from the models and converts it into plain text (a string). This makes it easier to use this output in applications, like displaying it to users or processing it further for other purposes.


In [5]:
chain.invoke({"politician": "jfk"})

"A curious fact about JFK is that he was a collector of maritime artifacts and spent much of his free time sailing on his family's boat, the Victura. He was also known to have a fascination with naval history and often visited maritime museums."

# Runnable execution order

![alt text](../data/png/lcel-2.png)

## 3 ways to execute chains: Invoke, Stream, Batch

In [6]:
Model = ChatOpenAI(model="gpt-3.5-turbo-0125")
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | Model

.invoke() call the chain on an input

In [7]:
chain.invoke({"topic": "woke"})

AIMessage(content='Why did the woke person bring a ladder to the protest? Because they wanted to take their activism to a higher level!', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 13, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-770f34e4-d1a3-4064-8480-d8cdef857e84-0', usage_metadata={'input_tokens': 13, 'output_tokens': 24, 'total_tokens': 37})

.stream() call the chain on an input and stream back chunks of the response

In [8]:
for s in chain.stream({"topic", "woke"}):
    print(s.content, end="", flush=True)

Why did the woke activist bring a pillow to the debate? Because they wanted to make sure everyone was staying on topic!

This for loop is used to show responses piece by piece as they are received and prints out responses immediately.

* `end=""`: This parameter ensures that each piece of content is added without adding a new line.
* `flush=True`: This parameter forces the output buffer to be flushed immediately after each print statement, ensuring that each piece of content is displayed to the user as soon as it is received witohut any delay.

.batch() is called when there is a list of inputs

In [9]:
chain.batch([{"topic":"cheese"}, {"topic": "wednesday"}])

[AIMessage(content='Why did the cheese go to the art exhibit? Because it wanted to see the "brie"lent artwork!', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 13, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4ee5bace-58c0-45f3-addc-1d65ef7aab41-0', usage_metadata={'input_tokens': 13, 'output_tokens': 24, 'total_tokens': 37}),
 AIMessage(content='Why did Wednesday go to the therapist? Because it had a case of the mid-week blues!', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 14, 'total_tokens': 33}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-63f4c235-8413-43ec-a089-eaeeffda50eb-0', usage_metadata={'input_tokens': 14, 'output_tokens': 19, 'total_tokens': 33})]

# Built-in Runnables

* RunnablePassthrough.
* RunnableLambda.
* RunnableParallel
    * itemgetter
* RunnableBranch

## RunnablePassthrough
   
   * It does not do anything to the input data
   * will output the original input without any modification

In [10]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()
chain.invoke("Abram")

'Abram'

## RunnableLambda

 * To use a custom function inside a LCEL 

In [11]:
def russian_lastname(name:str) -> str:
    return f"{name}ovich"

In [12]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

chain.invoke("Abram")

'Abramovich'

## RunnableParallel
  
  * For running tasks in parallel
  * Probably the most important and useful runnable

In [13]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)
chain.invoke("Abram")

{'operation_a': 'Abram', 'operation_b': 'Abramovich'}

In [14]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": lambda x : x["name"] + "ovich"
    }
)

chain.invoke({
    "name1": "Jordan",
    "name": "Abram"
})


{'operation_a': {'name1': 'Jordan', 'name': 'Abram'},
 'operation_b': 'Abramovich'}

We can add more runnables to the chain

In [15]:
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")
output_parser = StrOutputParser()

In [16]:
def russian_lastname_from_dictioanry(person):
    return person['name'] + "ovich"

In [17]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictioanry),
        "operation_c": RunnablePassthrough()
    }
) | prompt | Model | output_parser

In [18]:
chain.invoke({
    "name1": "Jordan",
    "name": "Abram"
})


"One curious fact about Roman Abramovich is that he once owned the world's largest yacht, the Eclipse. The yacht is equipped with two helicopter pads, a submarine, and a missile defense system, making it one of the most luxurious and secure vessels in the world. Abramovich reportedly spent over $1.5 billion on the yacht, showcasing his extravagant lifestyle and wealth."

In [19]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["AI Accelera has trained more than 10,000 alumni from all continents and top companies"], embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template=template)

model = ChatOpenAI(model="gpt-3.5-turbo")

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("who are the alumni of AI Accelera")


'The alumni of AI Accelera are individuals from all continents and top companies.'

#### Important: the syntax of RunnableParallel can have several variations.
* When composing a RunnableParallel with another Runnable you do not need to wrap it up in the RunnableParallel class. Inside a chain, the next three syntaxs are equivalent:
    * `RunnableParallel({"context": retriever, "question": RunnablePassthrough()})`
    * `RunnableParallel(context=retriever, question=RunnablePassthrough())`
    * `{"context": retriever, "question": RunnablePassthrough()}`

## Using itemgetter with RunnableParallel
* When you are calling the LLM with several different input variables.

In [58]:
from operator import itemgetter

vectorstore = FAISS.from_texts(
    ["AI Accelera has trained more than 5,000 Enterprise Alumni."], embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

model = ChatOpenAI(model="gpt-3.5-turbo")

prompt = ChatPromptTemplate.from_template(template=template)

chain = ( 
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language")
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "How many enterprise Alumni has trained AI Accelera?", "language": "Pirate English"})

'Arrr, AI Accelera has trained more than 5,000 Enterprise Alumni.'

## RunnableBranch: Router Chain
* A RunnableBranch is a special type of runnable that allows you to define a set of conditions and runnables to execute based on the input.
* **A RunnableBranch is initialized with a list of (condition, runnable) pairs and a default runnable**. It selects which branch by passing each condition the input it's invoked with. It selects the first condition to evaluate to True, and runs the corresponding runnable to that condition with the input.
* For advanced uses, a [custom function](https://python.langchain.com/v0.1/docs/expression_language/how_to/routing/) may be a better alternative than RunnableBranch.

The following advanced example can classify and respond to user questions based on specific topics like rock, politics, history, sports, or general inquiries. **It uses some new topics that we will explain in the following lesson**. Here’s a simplified explanation of each part:

1. **Prompt Templates**: Each template is tailored for a specific topic:
   - **rock_template**: Configured for rock and roll related questions.
   - **politics_template**: Tailored to answer questions about politics.
   - **history_template**: Designed for queries related to history.
   - **sports_template**: Set up to address sports-related questions.
   - **general_prompt**: A general template for queries that don't fit the specific categories.

   Each template includes a placeholder `{input}` where the actual user question will be inserted.

2. **RunnableBranch**: This is a branching mechanism that selects which template to use based on the topic of the question. It evaluates conditions (like `x["topic"] == "rock"`) to determine the topic and uses the appropriate prompt template.

3. **Topic Classifier**: A Pydantic class that classifies the topic of a user's question into one of the predefined categories (rock, politics, history, sports, or general).

4. **Classifier Chain**:
   - **Chain**: Processes the user's input to predict the topic.
   - **Parser**: Extracts the predicted topic from the classifier's output.

5. **RunnablePassthrough**: This component feeds the user's input and the classified topic into the RunnableBranch.

6. **Final Chain**:
   - The user's input is first processed to classify its topic.
   - The appropriate prompt is then selected based on the classified topic.
   - The selected prompt is used to formulate a question which is then sent to a model (like ChatOpenAI).
   - The model’s response is parsed as a string and returned.

7. **Execution**:
   - The chain is invoked with a sample question, "Who was Napoleon Bonaparte?" 
   - Based on the classification, it selects the appropriate template, generates a query to the chat model, and processes the response.

The system effectively creates a dynamic response generator that adjusts the way it answers based on the topic of the inquiry, making use of specialized knowledge for different subjects.

In [59]:
from langchain.prompts import PromptTemplate

rock_template = """You are a very smart rock and roll professor. \
You are great at answering questions about rock and roll in a concise\
and easy to understand manner.

Here is a question:
{input}"""

rock_prompt = PromptTemplate.from_template(rock_template)

politics_template = """You are a very good politics professor. \
You are great at answering politics questions..

Here is a question:
{input}"""

politics_prompt = PromptTemplate.from_template(politics_template)

history_template = """You are a very good history teacher. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods.

Here is a question:
{input}"""

history_prompt = PromptTemplate.from_template(history_template)

sports_template = """ You are a sports teacher.\
You are great at answering sports questions.

Here is a question:
{input}"""

sports_prompt = PromptTemplate.from_template(sports_template)

In [60]:
from langchain.schema.runnable import RunnableBranch

general_prompt = PromptTemplate.from_template(
    "You are a helpful assistant. Answer the question as accurately as you can.\n\n{input}"
)
prompt_branch = RunnableBranch(
  (lambda x: x["topic"] == "rock", rock_prompt),
  (lambda x: x["topic"] == "politics", politics_prompt),
  (lambda x: x["topic"] == "history", history_prompt),
  (lambda x: x["topic"] == "sports", sports_prompt),
  general_prompt
)

In [61]:
from typing import Literal

from langchain.pydantic_v1 import BaseModel
from langchain.output_parsers.openai_functions import PydanticAttrOutputFunctionsParser
from langchain_core.utils.function_calling import convert_to_openai_function

class TopicClassifier(BaseModel):
    "Classify the topic of the user question"
    
    topic: Literal["rock", "politics", "history", "sports"]
    "The topic of the user question. One of 'rock', 'politics', 'history', 'sports' or 'general'."

classifier_function = convert_to_openai_function(TopicClassifier)

llm = ChatOpenAI().bind(functions=[classifier_function], function_call={"name": "TopicClassifier"}) 

parser = PydanticAttrOutputFunctionsParser(pydantic_schema=TopicClassifier, attr_name="topic")

classifier_chain = llm | parser

The `classifier_function` classifies or categorizes the topic of a user's question into specific categories such as "rock," "politics," "history," or "sports." Here’s how it works in simple terms:

1. **Conversion to Function**: It converts the `TopicClassifier` Pydantic class, which is a predefined classification system, into a function that can be easily used with LangChain. This conversion process involves wrapping the class so that it can be integrated and executed within an OpenAI model.

2. **Topic Detection**: When you input a question, this function analyzes the content of the question to determine which category or topic it belongs to. It looks for keywords or patterns that match specific topics. For example, if the question is about a rock band, the classifier would identify the topic as "rock."

3. **Output**: The function outputs the identified topic as a simple label, like "rock" or "history." This label is then used by other parts of the LangChain to decide how to handle the question, such as choosing the right template for formulating a response.

In essence, the `classifier_function` acts as a smart filter that helps the system understand what kind of question is being asked so that it can respond more accurately and relevantly.

In [63]:
from operator import itemgetter

from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


final_chain = (
    RunnablePassthrough.assign(topic=itemgetter("input") | classifier_chain) 
    | prompt_branch 
    | ChatOpenAI()
    | StrOutputParser()
)

final_chain.invoke(
    {"input": "Who was Napoleon Bonaparte?"}
)

# Built-in functions for Runnables

## Use of .bind() to add argumetns to a Runnable in a LCEL Chain

In [20]:
prompt = ChatPromptTemplate.from_template("Tell me a curious fact about {soccer_player}")
output_parser = StrOutputParser()

In [25]:
chain = prompt | model.bind(stop=["Ronaldo"]) | output_parser
chain.invoke({"soccer_player": "Ronaldo"})

'One curious fact about Cristiano '